In [1]:
pip install dash dash-bootstrap-components plotly pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, dash_table
import dash_bootstrap_components as dbc
from datetime import datetime

# Load data
attendance = pd.read_csv('dr_free_fitness_attendance (3).csv')
members = pd.read_csv('dr_free_fitness_members.csv')
payment = pd.read_csv('dr_free_fitness_payments (3).csv')

# Cleaning
attendance['date'] = pd.to_datetime(attendance['date'])
members['join_date'] = pd.to_datetime(members['join_date'])
members['subscription_end_date'] = pd.to_datetime(members['subscription_end_date'])
payment['payment_date'] = pd.to_datetime(payment['payment_date'])
members['trainer_type'] = members['trainer_type'].fillna('No Trainer').replace('None', 'No Trainer')

# Master dataset
attendance_count = attendance.groupby('member_id').size().reset_index(name='attendance_count')
master = members.merge(payment, on='member_id').merge(attendance_count, on='member_id', how='left')
master['attendance_count'] = master['attendance_count'].fillna(0).astype(int)

today = pd.to_datetime('2026-01-11')
master['days_to_expiry'] = (master['subscription_end_date'] - today).dt.days

# Pre-calculate key metrics (done once)
total_revenue = master['amount'].sum()
potential_gap = master[master['membership_status'].isin(['Expired', 'Grace'])]['amount'].sum()
gap_percentage = (potential_gap / total_revenue * 100) if total_revenue > 0 else 0

# App setup - BRIGHT & CLEAN theme (recommended for presentation)
app = Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])



app.layout = dbc.Container(fluid=True, children=[
    
# Branded Header with Logo
html.Div([
    dbc.Row([
        dbc.Col(
            html.Img(
                src="/assets/logo.png",
                height="90px",
                alt="Dr Dre Fitness Logo",
                style={"marginRight": "25px", "borderRadius": "8px"}
            ),
            width="auto"
        ),
        dbc.Col([
            html.H1("Dr Dre Fitness",
                    style={'color': '#2c3e50', 'margin': '0', 'fontWeight': 'bold'}),
            html.H4("Executive Dashboard • Real-Time Insights",
                    style={'color': '#18bc9c', 'margin': '5px 0 0 0'})
        ], style={'paddingTop': '10px'})
    ], align="center", justify="start", className="mb-3"),
    
    html.Hr(style={'borderColor': '#18bc9c', 'margin': '10px 0 20px 0'})
], style={'padding': '20px 30px', 'backgroundColor': '#f8f9fa'}),

    dbc.Tabs([
        dbc.Tab(label="Overview", children=[
            dbc.Row([
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("Total Members", className="card-title"),
                    html.H3(f"{len(master):,}", className="card-text text-primary")
                ]), className="shadow"), width=2),
                
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("Active Members", className="card-title"),
                    html.H3(f"{len(master[master['membership_status']=='Active']):,}", className="card-text text-success")
                ]), className="shadow"), width=2),
                
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("Total Revenue", className="card-title"),
                    html.H3(f"₦{total_revenue:,.0f}", className="card-text text-info")
                ]), className="shadow"), width=2),
                
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("Avg Sessions", className="card-title"),
                    html.H3(f"{master['attendance_count'].mean():.1f}", className="card-text")
                ]), className="shadow"), width=2),
                
                # ← New: Lost Revenue Opportunity
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("Lost Revenue Opportunity", className="card-title text-warning"),
                    html.H3(f"₦{potential_gap:,.0f}", className="card-text text-danger"),
                    html.P(f"({gap_percentage:.1f}% of total)", className="small text-muted")
                ]), color="warning", outline=True, className="shadow"), width=4),
            ], className="mb-4 g-4"),

            dbc.Row([
                dbc.Col(dcc.Graph(id='status-pie'), width=6),
                dbc.Col(dcc.Graph(id='plan-bar'), width=6),
            ])
        ]),
        
        dbc.Tab(label="Revenue Insights", children=[
            dcc.Graph(id='monthly-revenue'),
            dcc.Graph(id='trainer-revenue'),
        ]),
        
        dbc.Tab(label="Attendance & Churn", children=[
            dcc.Dropdown(
                id='session-filter',
                options=[{'label': x, 'value': x} for x in ['All', 'Morning', 'Evening']],
                value='All',
                className="mb-3 w-50"
            ),
            dcc.Graph(id='attendance-trend'),
            dcc.Graph(id='churn-risk'),
        ]),
        
        dbc.Tab(label="Renewal Alerts", children=[
            html.H4("Members Expiring Soon (Next 30 Days)", className="mt-4 mb-3 text-primary"),
            dash_table.DataTable(
                id='expiring-table',
                columns=[{"name": i, "id": i} for i in ['full_name', 'member_id', 'subscription_plan', 'subscription_end_date', 'days_to_expiry']],
                style_cell={'textAlign': 'left'},
                style_header={'backgroundColor': '#18bc9c', 'fontWeight': 'bold', 'color': 'white'},
                style_data={'backgroundColor': '#f8f9fa'},
                page_size=12,
                style_as_list_view=True,
            )
        ]),
    ], className="mt-3")
])

@app.callback(
    [Output('status-pie', 'figure'),
     Output('plan-bar', 'figure'),
     Output('monthly-revenue', 'figure'),
     Output('trainer-revenue', 'figure'),
     Output('attendance-trend', 'figure'),
     Output('churn-risk', 'figure'),
     Output('expiring-table', 'data')],
    [Input('session-filter', 'value')]
)
def update_charts(session_filter):
    # Status Pie
    status_counts = master['membership_status'].value_counts()
    pie = px.pie(
        values=status_counts.values,
        names=status_counts.index,
        title="Membership Status Distribution",
        color_discrete_sequence=px.colors.qualitative.Set2
    )
    pie.update_layout(
        title_font=dict(size=20, color='#2c3e50', family='Arial Black'),
        xaxis_title=None,
        yaxis_title=None
    )

    # Top 10 Plans
    plans = master['subscription_plan'].value_counts().head(10)
    bar = px.bar(
        x=plans.index,
        y=plans.values,
        title="Top 10 Subscription Plans",
        color_discrete_sequence=px.colors.sequential.Teal
    )
    bar.update_layout(
        title_font=dict(size=20, color='#2c3e50', family='Arial Black'),
        xaxis={'showticklabels': False, 'title': None},
        yaxis={'showticklabels': False, 'title': None},
        showlegend=False,
        margin=dict(l=20, r=20, t=60, b=20),
        hovermode='x unified'
    )

    # Monthly Revenue Trend
    payment['month'] = payment['payment_date'].dt.strftime('%Y-%m')
    monthly = payment.groupby('month')['amount'].sum().reset_index()
    rev_line = px.line(
        monthly,
        x='month',
        y='amount',
        title="Monthly Revenue Trend",
        markers=True,
        color_discrete_sequence=['#18bc9c']
    )
    rev_line.update_layout(
        title_font=dict(size=20, color='#2c3e50', family='Arial Black'),
        xaxis={'showticklabels': False, 'title': None},
        yaxis={'showticklabels': False, 'title': None},
        showlegend=False,
        margin=dict(l=20, r=20, t=60, b=20),
        hovermode='x unified'
    )

    # Trainer Revenue
    trainer_rev = master.groupby('trainer_type')['amount'].sum().reset_index()
    trainer_pie = px.pie(
        trainer_rev,
        values='amount',
        names='trainer_type',
        title="Revenue by Trainer Type",
        color_discrete_sequence=px.colors.qualitative.Pastel
    )
    trainer_pie.update_layout(
        title_font=dict(size=20, color='#2c3e50', family='Arial Black')
    )

    # Daily Attendance
    att_filtered = attendance if session_filter == 'All' else attendance[attendance['session'] == session_filter]
    att_trend = att_filtered.groupby('date').size().reset_index(name='count')
    att_line = px.line(
        att_trend,
        x='date',
        y='count',
        title=f"Daily Attendance ({session_filter})",
        color_discrete_sequence=['#3498db']
    )
    att_line.update_layout(
        title_font=dict(size=20, color='#2c3e50', family='Arial Black'),
        xaxis={'showticklabels': False, 'title': None},
        yaxis={'showticklabels': False, 'title': None},
        showlegend=False,
        margin=dict(l=20, r=20, t=60, b=20),
        hovermode='x unified'
    )

    # Churn Risk
    churn = master[['full_name', 'attendance_count', 'membership_status']]\
              .sort_values('attendance_count').head(20)
    churn_bar = px.bar(
        churn,
        x='full_name',
        y='attendance_count',
        color='membership_status',
        title="Top 20 Lowest Attendance (Churn Risk)",
        color_discrete_sequence=px.colors.qualitative.Set1
    )
    churn_bar.update_layout(
        title_font=dict(size=20, color='#2c3e50', family='Arial Black'),
        xaxis={'showticklabels': False, 'title': None},
        yaxis={'showticklabels': False, 'title': None},
        margin=dict(l=20, r=20, t=60, b=20),
        hovermode='x unified'
    )

    # Expiring Table
    expiring = master[
        (master['days_to_expiry'] > 0) & (master['days_to_expiry'] <= 30)
    ][['full_name', 'member_id', 'subscription_plan', 'subscription_end_date', 'days_to_expiry']]\
     .sort_values('days_to_expiry')
    
    return pie, bar, rev_line, trainer_pie, att_line, churn_bar, expiring.to_dict('records')

if __name__ == '__main__':
    app.run(debug=True)

server = app.server


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, dash_table
import dash_bootstrap_components as dbc
from datetime import datetime

# Load data
attendance = pd.read_csv('dr_free_fitness_attendance (3).csv')
members = pd.read_csv('dr_free_fitness_members.csv')
payment = pd.read_csv('dr_free_fitness_payments (3).csv')

# Cleaning
attendance['date'] = pd.to_datetime(attendance['date'])
members['join_date'] = pd.to_datetime(members['join_date'])
members['subscription_end_date'] = pd.to_datetime(members['subscription_end_date'])
payment['payment_date'] = pd.to_datetime(payment['payment_date'])
members['trainer_type'] = members['trainer_type'].fillna('No Trainer').replace('None', 'No Trainer')

# Master dataset
attendance_count = attendance.groupby('member_id').size().reset_index(name='attendance_count')
master = members.merge(payment, on='member_id').merge(attendance_count, on='member_id', how='left')
master['attendance_count'] = master['attendance_count'].fillna(0).astype(int)

today = pd.to_datetime('2026-01-11')
master['days_to_expiry'] = (master['subscription_end_date'] - today).dt.days

# Pre-calculate key metrics
total_revenue = master['amount'].sum()
potential_gap = master[master['membership_status'].isin(['Expired', 'Grace'])]['amount'].sum()
gap_percentage = (potential_gap / total_revenue * 100) if total_revenue > 0 else 0

# App setup
app = Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

app.layout = dbc.Container(fluid=True, children=[
    
    # Branded Header with Logo
    html.Div([
        dbc.Row([
            dbc.Col(
                html.Img(
                    src="/assets/logo.png",                  
                    height="90px",                      
                    alt="DR Free Fitness Logo",
                    style={"marginRight": "25px", "borderRadius": "8px"}
                ),
                width="auto"
            ),
            dbc.Col([
                html.H1("DR Free Fitness",
                        style={'color': '#2c3e50', 'margin': '0', 'fontWeight': 'bold'}),
                html.H4("Executive Dashboard • Real-Time Insights",
                        style={'color': '#18bc9c', 'margin': '5px 0 0 0'})
            ], style={'paddingTop': '10px'})
        ], align="center", justify="start", className="mb-3"),
        
        html.Hr(style={'borderColor': '#18bc9c', 'margin': '10px 0 20px 0'})
    ], style={'padding': '20px 30px', 'backgroundColor': '#f8f9fa'}),

    dbc.Tabs([
        dbc.Tab(label="Overview", children=[
            dbc.Row([
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("Total Members", className="card-title"),
                    html.H3(f"{len(master):,}", className="card-text text-primary")
                ]), className="shadow"), width=2),
                
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("Active Members", className="card-title"),
                    html.H3(f"{len(master[master['membership_status']=='Active']):,}", className="card-text text-success")
                ]), className="shadow"), width=2),
                
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("Total Revenue", className="card-title"),
                    html.H3(f"₦{total_revenue:,.0f}", className="card-text text-info")
                ]), className="shadow"), width=2),
                
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("Avg Sessions", className="card-title"),
                    html.H3(f"{master['attendance_count'].mean():.1f}", className="card-text")
                ]), className="shadow"), width=2),
                
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("Lost Revenue Opportunity", className="card-title text-warning"),
                    html.H3(f"₦{potential_gap:,.0f}", className="card-text text-danger"),
                    html.P(f"({gap_percentage:.1f}% of total)", className="small text-muted")
                ]), color="warning", outline=True, className="shadow"), width=4),
            ], className="mb-4 g-4"),

            dbc.Row([
                dbc.Col(dcc.Graph(id='status-pie'), width=6),
                dbc.Col(dcc.Graph(id='plan-bar'), width=6),
            ])
        ]),
        
        # ... (keep all your other tabs exactly the same)
        
        dbc.Tab(label="Revenue Insights", children=[
            dcc.Graph(id='monthly-revenue'),
            dcc.Graph(id='trainer-revenue'),
        ]),
        
        dbc.Tab(label="Attendance & Churn", children=[
            dcc.Dropdown(
                id='session-filter',
                options=[{'label': x, 'value': x} for x in ['All', 'Morning', 'Evening']],
                value='All',
                className="mb-3 w-50"
            ),
            dcc.Graph(id='attendance-trend'),
            dcc.Graph(id='churn-risk'),
        ]),
        
        dbc.Tab(label="Renewal Alerts", children=[
            html.H4("Members Expiring Soon (Next 30 Days)", className="mt-4 mb-3 text-primary"),
            dash_table.DataTable(
                id='expiring-table',
                columns=[{"name": i, "id": i} for i in ['full_name', 'member_id', 'subscription_plan', 'subscription_end_date', 'days_to_expiry']],
                style_cell={'textAlign': 'left'},
                style_header={'backgroundColor': '#18bc9c', 'fontWeight': 'bold', 'color': 'white'},
                style_data={'backgroundColor': '#f8f9fa'},
                page_size=12,
                style_as_list_view=True,
            )
        ]),
    ], className="mt-3")
])

@app.callback(
    [Output('status-pie', 'figure'),
     Output('plan-bar', 'figure'),
     Output('monthly-revenue', 'figure'),
     Output('trainer-revenue', 'figure'),
     Output('attendance-trend', 'figure'),
     Output('churn-risk', 'figure'),
     Output('expiring-table', 'data')],
    [Input('session-filter', 'value')]
)
def update_charts(session_filter):
    
    # Status Pie
    status_counts = master['membership_status'].value_counts()
    pie = px.pie(
        values=status_counts.values,
        names=status_counts.index,
        title="Membership Status Distribution",
        color_discrete_sequence=px.colors.qualitative.Set2
    )
    pie.update_layout(
        title_font=dict(size=20, color='#2c3e50', family='Arial Black'),
        xaxis_title=None,
        yaxis_title=None
    )

    # Top 10 Plans
    plans = master['subscription_plan'].value_counts().head(10)
    bar = px.bar(
        x=plans.index,
        y=plans.values,
        title="Top 10 Subscription Plans",
        color_discrete_sequence=px.colors.sequential.Teal
    )
    bar.update_layout(
        title_font=dict(size=20, color='#2c3e50', family='Arial Black'),
        xaxis={'showticklabels': False, 'title': None},
        yaxis={'showticklabels': False, 'title': None},
        showlegend=False,
        margin=dict(l=20, r=20, t=60, b=20),
        hovermode='x unified'
    )

    # Monthly Revenue Trend
    payment['month'] = payment['payment_date'].dt.strftime('%Y-%m')
    monthly = payment.groupby('month')['amount'].sum().reset_index()
    rev_line = px.line(
        monthly,
        x='month',
        y='amount',
        title="Monthly Revenue Trend",
        markers=True,
        color_discrete_sequence=['#18bc9c']
    )
    rev_line.update_layout(
        title_font=dict(size=20, color='#2c3e50', family='Arial Black'),
        xaxis={'showticklabels': False, 'title': None},
        yaxis={'showticklabels': False, 'title': None},
        showlegend=False,
        margin=dict(l=20, r=20, t=60, b=20),
        hovermode='x unified'
    )

    # Trainer Revenue
    trainer_rev = master.groupby('trainer_type')['amount'].sum().reset_index()
    trainer_pie = px.pie(
        trainer_rev,
        values='amount',
        names='trainer_type',
        title="Revenue by Trainer Type",
        color_discrete_sequence=px.colors.qualitative.Pastel
    )
    trainer_pie.update_layout(
        title_font=dict(size=20, color='#2c3e50', family='Arial Black')
    )

    # Daily Attendance
    att_filtered = attendance if session_filter == 'All' else attendance[attendance['session'] == session_filter]
    att_trend = att_filtered.groupby('date').size().reset_index(name='count')
    att_line = px.line(
        att_trend,
        x='date',
        y='count',
        title=f"Daily Attendance ({session_filter})",
        color_discrete_sequence=['#3498db']
    )
    att_line.update_layout(
        title_font=dict(size=20, color='#2c3e50', family='Arial Black'),
        xaxis={'showticklabels': False, 'title': None},
        yaxis={'showticklabels': False, 'title': None},
        showlegend=False,
        margin=dict(l=20, r=20, t=60, b=20),
        hovermode='x unified'
    )

    # Churn Risk
    churn = master[['full_name', 'attendance_count', 'membership_status']]\
              .sort_values('attendance_count').head(20)
    churn_bar = px.bar(
        churn,
        x='full_name',
        y='attendance_count',
        color='membership_status',
        title="Top 20 Lowest Attendance (Churn Risk)",
        color_discrete_sequence=px.colors.qualitative.Set1
    )
    churn_bar.update_layout(
        title_font=dict(size=20, color='#2c3e50', family='Arial Black'),
        xaxis={'showticklabels': False, 'title': None},
        yaxis={'showticklabels': False, 'title': None},
        margin=dict(l=20, r=20, t=60, b=20),
        hovermode='x unified'
    )

    # Expiring Table
    expiring = master[
        (master['days_to_expiry'] > 0) & (master['days_to_expiry'] <= 30)
    ][['full_name', 'member_id', 'subscription_plan', 'subscription_end_date', 'days_to_expiry']]\
     .sort_values('days_to_expiry')
    
    return pie, bar, rev_line, trainer_pie, att_line, churn_bar, expiring.to_dict('records')

if __name__ == '__main__':
    app.run(debug=True)